# Proprietary analysis 1: average distance between operational bases and gas stations

The objective here is to analyze: There is a key differentiator between majors in the fuel distribution logistics?

Expected answer: No.

### Limitations:

- For some reason, the code is ignoring gas stations within the 50 km radius around the operational bases.
- In this code, I can't seggregate the gas stations. As a result, the gas stations around some Vibra's base that are being used for the calculation could not be BR flaggled.

In [1]:
%pip install geopy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import googlemaps  # Import the googlemaps library to interact with the Google Maps API
from math import sqrt  # Import the sqrt function from the math module to calculate Euclidean distance
import folium  # Import the folium library for creating maps
from folium.plugins import MarkerCluster  # Import the MarkerCluster plugin for clustering markers
import time

In [2]:
data = pd.read_excel(r'G:\Meu Drive\Vida\UNICAMP\GMF\Challenges\CFA 2024\VBBR3\Modelagem - VBBR3\Bases Operacionais VBBR UGPA RAIZ.xlsx')
data

,Razão Social,Identificação da Base,Capacidade Total,Complemento,Latitude,Longitude,Região
0,UGPA,NSC,27732.20,Base de São Caetano,-23.611517,-46.554979,Sudeste
1,UGPA,BBU,6065.22,Base de Bauru,-22.317719,-49.051082,Sudeste
2,UGPA,NaN,2977.50,Base de Bagé,-31.344773,-54.077365,Sul
3,UGPA,BCA,5948.85,Base de Cruz Alta,-28.682478,-53.613789,Sul
4,UGPA,BSM,5872.00,Base de Santa Maria,-29.693512,-53.768165,Sul
...,...,...,...,...,...,...,...
100,VBBR,BAMON,3247.33,Base de Belo Monte,-3.124258,-51.721502,Norte
101,VBBR,BEAPE,126665.00,Base de Suape,-8.400322,-34.964672,Nordeste
102,VBBR,BACAI,10958.00,Base de Açaílândia,-4.902363,-47.394740,Nordeste
103,VBBR,BALON,20176.16,Base de Londrina,-23.277903,-51.211789,Sul


In [12]:
# Initialize the Google Maps client with your API key
with open('api_key.txt', 'r') as file:
    api_key = file.read()
gmaps = googlemaps.Client(key=api_key)  # Create a Google Maps client using the provided API key

# Function to find gas stations around a given latitude and longitude with multiple radius
def find_gas_stations(lat, lng, radii):
    all_results = []  # Initialize an empty list to store all results
    
    for radius in radii:  # Iterate over each radius
        places_result = gmaps.places_nearby(location=(lat, lng), radius=radius, type='gas_station')
        results = places_result['results']  # Initialize results with the first page of results
        
        # Handle pagination to get all results
        while 'next_page_token' in places_result:
            next_page_token = places_result['next_page_token']
            time.sleep(2)  # Add a delay to handle rate limits
            places_result = gmaps.places_nearby(location=(lat, lng), radius=radius, type='gas_station', page_token=next_page_token)
            results.extend(places_result['results'])  # Append the next page of results to the existing results
        
        all_results.extend(results)  # Append the results for the current radius to all_results
    
    return all_results  # Return the aggregated results

In [14]:
nearby_gas_stations = data.apply(lambda row: find_gas_stations(row['Latitude'], row['Longitude'], radii=[5000, 10000, 25000]), axis=1)  # Find nearby gas stations for each base with a specified radius
nearby_gas_stations

0      [{'business_status': 'OPERATIONAL', 'geometry'...
1      [{'business_status': 'OPERATIONAL', 'geometry'...
2      [{'business_status': 'OPERATIONAL', 'geometry'...
3      [{'business_status': 'OPERATIONAL', 'geometry'...
4      [{'business_status': 'OPERATIONAL', 'geometry'...
                             ...                        
100    [{'business_status': 'OPERATIONAL', 'geometry'...
101    [{'business_status': 'OPERATIONAL', 'geometry'...
102    [{'business_status': 'OPERATIONAL', 'geometry'...
103    [{'business_status': 'OPERATIONAL', 'geometry'...
104    [{'business_status': 'OPERATIONAL', 'geometry'...
Length: 105, dtype: object

In [15]:
def calculate_euclidean_distance(base_lat, base_lng, station_lat, station_lng):
    return sqrt((base_lat - station_lat)**2 + (base_lng - station_lng)**2)  # Calculate Euclidean distance between base and station

# Calculate distances for each base and its nearby gas stations
distances = []  # Initialize an empty list to store distances

for index, row in data.iterrows():  # Iterate over each row in the data DataFrame
    base_lat = row['Latitude']  # Get the latitude of the base
    base_lng = row['Longitude']  # Get the longitude of the base
    stations = nearby_gas_stations[index]  # Get the nearby gas stations for the current base
    
    for station in stations:  # Iterate over each gas station
        station_lat = station['geometry']['location']['lat']  # Get the latitude of the gas station
        station_lng = station['geometry']['location']['lng']  # Get the longitude of the gas station
        distance = calculate_euclidean_distance(base_lat, base_lng, station_lat, station_lng)  # Calculate the distance between the base and the gas station
        distances.append({  # Append the distance information to the distances list
            'Company': row['Razão Social'],  # Add the company name
            'Base': row['Identificação da Base'],  # Add the base identification
            'Region': row['Região'],  # Add the region
            'Base Latitude': base_lat,  # Add the base latitude
            'Base Longitude': base_lng,  # Add the base longitude
            'Station Latitude': station_lat,  # Add the station latitude
            'Station Longitude': station_lng,  # Add the station longitude
            'Distance': distance  # Add the calculated distance
        })

# Convert the distances list to a DataFrame for better visualization
distances_df = pd.DataFrame(distances)  # Create a DataFrame from the distances list
distances_df  # Display the DataFrame

,Company,Base,Region,Base Latitude,Base Longitude,Station Latitude,Station Longitude,Distance
0,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.577770,-46.542080,0.036128
1,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.596677,-46.536834,0.023440
2,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.624297,-46.540862,0.019043
3,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.629125,-46.567665,0.021702
4,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.591092,-46.569447,0.025029
...,...,...,...,...,...,...,...,...
15562,VBBR,BAVIC,Norte,-1.541935,-48.742751,-1.673375,-48.627612,0.174738
15563,VBBR,BAVIC,Norte,-1.541935,-48.742751,-1.397557,-48.854334,0.182471
15564,VBBR,BAVIC,Norte,-1.541935,-48.742751,-1.398819,-48.864918,0.188167
15565,VBBR,BAVIC,Norte,-1.541935,-48.742751,-1.398782,-48.864918,0.188196


In [ ]:
# Initialize the map centered around the average latitude and longitude of the bases
map_center = [data['Latitude'].mean(), data['Longitude'].mean()]  # Calculate the center of the map
mymap = folium.Map(location=map_center, zoom_start=6)  # Create a folium map centered at map_center

# Create a marker cluster to group markers
marker_cluster = MarkerCluster().add_to(mymap)  # Add a marker cluster to the map

# Add markers and lines to the map
for index, row in distances_df.iterrows():  # Iterate over each row in the distances DataFrame
    base_location = [row['Base Latitude'], row['Base Longitude']]  # Get the base location
    station_location = [row['Station Latitude'], row['Station Longitude']]  # Get the gas station location
    
    # Add marker for the base
    folium.Marker(
        location=base_location,
        popup=f"Base: {row['Base']}<br>Company: {row['Company']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)  # Add a marker for the base to the marker cluster
    
    # Add marker for the gas station
    folium.Marker(
        location=station_location,
        popup=f"Gas Station<br>Distance: {row['Distance']:.2f} units",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(marker_cluster)  # Add a marker for the gas station to the marker cluster
    
    # Determine the line color based on the company
    if row['Company'] == 'VBBR':
        line_color = '#033F19'
    elif row['Company'] == 'RAIZ':
        line_color = '#791877'
    elif row['Company'] == 'UGPA':
        line_color = '#0041BD'
    
    # Draw a line between the base and the gas station
    folium.PolyLine(
        locations=[base_location, station_location],
        color=line_color,
        weight=2.5,
        opacity=1
    ).add_to(mymap)  # Add a line connecting the base and the gas station to the map

# Save the map to an HTML file
mymap.save('bases_gas_stations_map.html')  # Save the map as an HTML file

# Display the map in a Jupyter Notebook (if applicable)
mymap  # Display the map in the notebook

In [17]:
from geopy.distance import geodesic  # Import the geodesic function from the geopy.distance module

# Function to convert Euclidean distance in degrees to kilometers
def convert_to_kilometers(base_lat, base_lng, station_lat, station_lng):
    return geodesic((base_lat, base_lng), (station_lat, station_lng)).kilometers  # Calculate the geodesic distance between two points and return it in kilometers

# Apply the conversion to the distances DataFrame
distances_df['Distance (kilometers)'] = distances_df.apply(
    lambda row: convert_to_kilometers(row['Base Latitude'], row['Base Longitude'], row['Station Latitude'], row['Station Longitude']),
    axis=1  # Apply the conversion function to each row of the DataFrame
)

distances_df  # Display the updated DataFrame

,Company,Base,Region,Base Latitude,Base Longitude,Station Latitude,Station Longitude,Distance,Distance (kilometers)
0,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.577770,-46.542080,0.036128,3.962674
1,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.596677,-46.536834,0.023440,2.476010
2,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.624297,-46.540862,0.019043,2.019652
3,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.629125,-46.567665,0.021702,2.340714
4,UGPA,NSC,Sudeste,-23.611517,-46.554979,-23.591092,-46.569447,0.025029,2.701348
...,...,...,...,...,...,...,...,...,...
15562,VBBR,BAVIC,Norte,-1.541935,-48.742751,-1.673375,-48.627612,0.174738,19.374982
15563,VBBR,BAVIC,Norte,-1.541935,-48.742751,-1.397557,-48.854334,0.182471,20.225161
15564,VBBR,BAVIC,Norte,-1.541935,-48.742751,-1.398819,-48.864918,0.188167,20.862875
15565,VBBR,BAVIC,Norte,-1.541935,-48.742751,-1.398782,-48.864918,0.188196,20.866008


In [18]:
# Calculate the average distance grouped by company
average_distance_by_company = distances_df.groupby('Company')['Distance (kilometers)'].mean()  # Group the distances DataFrame by 'Company' and calculate the mean of 'Distance (km)' for each group

# Calculate the median distance grouped by company
median_distance_by_company = distances_df.groupby('Company')['Distance (kilometers)'].median()  # Group the distances DataFrame by 'Company' and calculate the median of 'Distance (km)' for each group

average_distance_by_company, median_distance_by_company  # Display the resulting Series with the average and median distances by company

(Company
 RAIZ    5.333585
 UGPA    4.681751
 VBBR    5.326573
 Name: Distance (kilometers), dtype: float64,
 Company
 RAIZ    3.725543
 UGPA    3.645137
 VBBR    3.936279
 Name: Distance (kilometers), dtype: float64)

In [19]:
# Calculate the average distance grouped by company and region
average_distance_by_company_region = distances_df.groupby(['Company', 'Region'])['Distance (kilometers)'].mean()
median_distance_by_company_region = distances_df.groupby(['Company', 'Region'])['Distance (kilometers)'].median()  # Calculate the mean distance by company and region

average_distance_by_company_region, median_distance_by_company_region  # Display the resulting Series with the average distances by company and region

(Company  Region      
 RAIZ     Centro Oeste     7.042840
          Nordeste         4.894462
          Norte           18.396476
          Sudeste          5.060524
          Sul              3.533550
 UGPA     Centro Oeste     8.095882
          Nordeste         3.873354
          Norte            5.050568
          Sudeste          4.336144
          Sul              4.320197
 VBBR     Centro Oeste     6.006790
          Nordeste         6.649890
          Norte            5.266642
          Sudeste          4.695761
          Sul              3.439671
 Name: Distance (kilometers), dtype: float64,
 Company  Region      
 RAIZ     Centro Oeste     7.023505
          Nordeste         3.694765
          Norte           23.230277
          Sudeste          3.670178
          Sul              2.373333
 UGPA     Centro Oeste     6.844248
          Nordeste         3.387793
          Norte            4.555537
          Sudeste          3.240682
          Sul              3.484687
 VBBR   